In [ ]:
import sys
sys.argv.append('--JOB_NAME')
sys.argv.append('address-cleaning')

sys.argv.append('--source_catalog_database')
sys.argv.append('housing-repairs-raw-zone')

sys.argv.append('--source_catalog_table')
sys.argv.append('housing_repairs_tv_aerials')

sys.argv.append('--cleaned_repairs_s3_bucket_target')
sys.argv.append('s3://dataplatform-stg-refined-zone/housing-repairs/repairs_electrical_mechanical_fire/housing-tv-aerials/cleaned')

In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.ml import Pipeline
from pyspark.ml.feature import RegexTokenizer, NGram, HashingTF, MinHashLSH
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col, trim, when, max, trim
import pyspark.sql.functions as F
from pyspark.sql.types import StringType
from awsglue.dynamicframe import DynamicFrame
import re

def get_glue_env_var(key, default="none"):
    if f'--{key}' in sys.argv:
        return getResolvedOptions(sys.argv, [key])[key]
    else:
        return default


def getLatestPartitions(dfa):
   dfa = dfa.where(col('import_year') == dfa.select(max('import_year')).first()[0])
   dfa = dfa.where(col('import_month') == dfa.select(max('import_month')).first()[0])
   dfa = dfa.where(col('import_day') == dfa.select(max('import_day')).first()[0])
   return dfa

def clean_column_names(df):
    df= df.select([F.col("`{0}`".format(c)).alias(c.replace('.', '')) for c in df.columns])
    df = df.select([F.col(col).alias(re.sub("_$","", col)) for col in df.columns]) 
    df2 = df.select([F.col(col).alias(re.sub("[^0-9a-zA-Z$]+","_", col.lower())) for col in df.columns]) 
    return df2

In [ ]:
args = getResolvedOptions(sys.argv, ['JOB_NAME'])

source_catalog_database = get_glue_env_var('source_catalog_database', '')
source_catalog_table    = get_glue_env_var('source_catalog_table', '')
cleaned_repairs_s3_bucket_target = get_glue_env_var('cleaned_repairs_s3_bucket_target', '')


sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
logger = glueContext.get_logger()
job = Job(glueContext)
job.init(args['JOB_NAME'], args)


In [ ]:
source_data = glueContext.create_dynamic_frame.from_catalog(
    name_space=source_catalog_database,
    table_name=source_catalog_table,
) 

df = source_data.toDF()
df = getLatestPartitions(df)

In [ ]:
df.printSchema()

In [ ]:
df2 = clean_column_names(df)

df2.printSchema()

In [ ]:
df2.show(vertical=True)

In [ ]:
df2 = df2.withColumn('data_source', F.lit('ElecMechFire - TV Aerials'))
df2 = df2.withColumn('datetime_raised', F.to_timestamp('date', 'yyyy-MM-dd HH:mm:ss'))
#rename column names to reflect harmonised column banes
df2 = df2.withColumn('status_of_completed_y_n', F.when(df2['status_of_completed_y_n']=='Y', 'Completed').otherwise(''))\

df2 = df2.withColumnRenamed('requested_by', 'operative') \
    .withColumnRenamed('address', 'property_address') \
    .withColumnRenamed('description', 'description_of_work') \
    .withColumnRenamed('priority_code', 'work_priority_description') \
    .withColumnRenamed('temp_order_number', 'temp_order_number_full') \
    .withColumnRenamed('cost_of_repairs_work', 'order_value') \
    .withColumnRenamed('status_of_completed_y_n', 'order_status') \


df2.printSchema()

In [ ]:
def map_repair_priority(code):
    if code == 'Immediate':
        return 1
    elif code == 'Emergency':
        return 2
    elif code == 'Urgent':
        return 3
    elif code == 'Normal':
        return 4
    else:
        return None

# # convert to a UDF Function by passing in the function and the return type of function (string in this case)
udf_map_repair_priority = F.udf(map_repair_priority, StringType())
# apply function
df2 = df2.withColumn('work_priority_priority_code', udf_map_repair_priority('work_priority_description'))

In [ ]:
df2=df2.select("data_source", "datetime_raised", 
               "operative", "property_address", "order_value",
               "description_of_work", "work_priority_description",
               "temp_order_number_full", "order_value", 
               "work_priority_priority_code",
               "import_datetime", "import_timestamp", "import_year",
               "import_month", "import_day", "import_date", 
                )

In [ ]:
df2.printSchema()
df2.show(vertical=True)

In [ ]:
cleanedDataframe = DynamicFrame.fromDF(df2, glueContext, "cleanedDataframe")
parquetData = glueContext.write_dynamic_frame.from_options(
    frame=cleanedDataframe,
    connection_type="s3",
    format="parquet",
    connection_options={"path": cleaned_repairs_s3_bucket_target,"partitionKeys": ["import_year", "import_month", "import_day", "import_date"]},
    transformation_ctx="parquetData")
job.commit()